In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))

    return epochs, pp_directory

In [ ]:
def direction_index(task, direction, pp):
    if task == 'aln':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\aln_MovementWorkspace_direction.csv')
    elif task == 'rdmrot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmrot_MovementWorkspace_direction.csv')
    elif task == 'rdmmir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmmir_MovementWorkspace_direction.csv')
    elif task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rot_MovementWorkspace_direction.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\mir_MovementWorkspace_direction.csv')
    
    ppdata = list(df.iloc[:,pp+1])

    index = [i for i,v in enumerate(ppdata) if direction == v]

    numtrials = len(index)
    
    return index, numtrials

In [ ]:
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

    
    return(triallist)

In [ ]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
      
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]

        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [ ]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, task, training):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_EarlyLate_%s_%s_block%s-epo.fif' % (pp_num, condname, task, training))
    data.save(out_fname, overwrite = True)

In [ ]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, task, training):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (pp_num, condname, task, training))
    evoked.save(out_fname, overwrite = True)

In [ ]:
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#ALIGNED
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], pp)
        aldata = epoch_trials('aln', pp, data, diridx)
        output_condition_epochs(pp, aldata, ppdir, conditionnames[cond], 'aligned', 'all')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, aldata, ppdir, conditionnames[cond], 'aligned', 'all')
        
#ALIGNED
#Separate according to corresponding participants for rotation and mirror (only half of N do each perturbation type)

aln_rot_participants = [0,1,2,3,8,9,10,11,16,17,18,19,24,25,26,27]
aln_mir_participants = [4,5,6,7,12,13,14,15,20,21,22,23,28,29,30,31]

#aligned for rotation
for pp in list(range(0,len(aln_rot_participants))):
    data, ppdir = load_epochs(aln_rot_participants[pp], root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], aln_rot_participants[pp])
        aldata = epoch_trials('aln', aln_rot_participants[pp], data, diridx)
        output_condition_epochs(aln_rot_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_rot', 'all')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(aln_rot_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_rot', 'all')
        
#aligned for mirror
for pp in list(range(0,len(aln_mir_participants))):
    data, ppdir = load_epochs(aln_mir_participants[pp], root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], aln_mir_participants[pp])
        aldata = epoch_trials('aln', aln_mir_participants[pp], data, diridx)
        output_condition_epochs(aln_mir_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_mir', 'all')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(aln_mir_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_mir', 'all')
        
#Separate according to corresponding participants for early and late in random (half for early, half for late)
aln_early_rdm_participants = [0,2,5,7,8,10,13,15,16,18,21,23,24,26,29,31]
aln_late_rdm_participants = [1,3,4,6,9,11,12,14,17,19,20,22,25,27,28,30]

#aligned for early rdm
for pp in list(range(0,len(aln_early_rdm_participants))):
    data, ppdir = load_epochs(aln_early_rdm_participants[pp], root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], aln_early_rdm_participants[pp])
        aldata = epoch_trials('aln', aln_early_rdm_participants[pp], data, diridx)
        output_condition_epochs(aln_early_rdm_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_early_rdm', 'all')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(aln_early_rdm_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_early_rdm', 'all')
        
#aligned for late rdm
for pp in list(range(0,len(aln_late_rdm_participants))):
    data, ppdir = load_epochs(aln_late_rdm_participants[pp], root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], aln_late_rdm_participants[pp])
        aldata = epoch_trials('aln', aln_late_rdm_participants[pp], data, diridx)
        output_condition_epochs(aln_late_rdm_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_late_rdm', 'all')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(aln_late_rdm_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_late_rdm', 'all')

In [ ]:
#get grand averages - ALIGNED
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']

#aligned
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in participants:
        id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], conditionnames[condition], 'aligned', 'all'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
    if condition == 0:
        aln_right_grand = mne.grand_average(evoked_list)
        aln_right_flist = evoked_list
    elif condition == 1:
        aln_left_grand = mne.grand_average(evoked_list)
        aln_left_flist = evoked_list
        
#aligned for rotation
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in list(range(0,len(aln_rot_participants))):
        id_directory = os.path.join(data_directory, 'p%03d/' % aln_rot_participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (aln_rot_participants[pp], conditionnames[condition], 'aln_rot', 'all'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
    if condition == 0:
        alnrot_right_grand = mne.grand_average(evoked_list)
        alnrot_right_flist = evoked_list
    elif condition == 1:
        alnrot_left_grand = mne.grand_average(evoked_list)
        alnrot_left_flist = evoked_list
        
#aligned for mirror
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in list(range(0,len(aln_rot_participants))):
        id_directory = os.path.join(data_directory, 'p%03d/' % aln_mir_participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (aln_mir_participants[pp], conditionnames[condition], 'aln_mir', 'all'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
    if condition == 0:
        alnmir_right_grand = mne.grand_average(evoked_list)
        alnmir_right_flist = evoked_list
    elif condition == 1:
        alnmir_left_grand = mne.grand_average(evoked_list)
        alnmir_left_flist = evoked_list
        
#aligned for early rdm
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in list(range(0,len(aln_early_rdm_participants))):
        id_directory = os.path.join(data_directory, 'p%03d/' % aln_early_rdm_participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (aln_early_rdm_participants[pp], conditionnames[condition], 'aln_early_rdm', 'all'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
    if condition == 0:
        alnearlyrdm_right_grand = mne.grand_average(evoked_list)
        alnearlyrdm_right_flist = evoked_list
    elif condition == 1:
        alnearlyrdm_left_grand = mne.grand_average(evoked_list)
        alnearlyrdm_left_flist = evoked_list
        
#aligned for late rdm
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in list(range(0,len(aln_late_rdm_participants))):
        id_directory = os.path.join(data_directory, 'p%03d/' % aln_late_rdm_participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (aln_late_rdm_participants[pp], conditionnames[condition], 'aln_late_rdm', 'all'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
    if condition == 0:
        alnlaterdm_right_grand = mne.grand_average(evoked_list)
        alnlaterdm_right_flist = evoked_list
    elif condition == 1:
        alnlaterdm_left_grand = mne.grand_average(evoked_list)
        alnlaterdm_left_flist = evoked_list

In [ ]:
#save as evoked objects
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#ROTATED and MIRROR TRAINING
tasks = ['rot', 'mir']

#split into training blocks
#early (first 45 trials), late (last 45 trials)
b0 = list(range(0,45))
b1 = list(range(45,90))

training = [b0, b1]

       
#rot and mir
for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for cond in range(0, len(conditions)):
            for t in range(0, len(training)):
                diridx, dirtrials = direction_index(tasks[task], conditions[cond], pp)
                diridx = [i for i in diridx if i in training[t]]
#                 print(diridx)
                if len(diridx) == 0:
                    break
                pdata = epoch_trials(tasks[task], pp, data, diridx)
#                 print(t, cond)
#                 print(pdata)
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], tasks[task], t)
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], tasks[task], t)

In [ ]:
#get grand averages - ROTATED AND MIRROR
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']
tasks = ['rot', 'mir']
training = [0, 1]
#rot and mir
for task in range(0, len(tasks)):
    for condition in range(0, len(conditionnames)):
        for t in range(0, len(training)):
            flist = []
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], conditionnames[condition], tasks[task], training[t]))
                if os.path.exists(fname) == False:
                    continue
#                 print(fname)
                evoked = mne.read_evokeds(fname)
                flist.append(evoked)
                evoked = evoked[0]
                evoked_list.append(evoked)
#                 print(evoked_list)
            if task == 0 and condition == 0 and t == 0:
                rot_right_b0_grand = mne.grand_average(evoked_list)
                rot_right_b0_flist = evoked_list
            elif task == 0 and condition == 0 and t == 1:
                rot_right_b1_grand = mne.grand_average(evoked_list)
                rot_right_b1_flist = evoked_list
            elif task == 0 and condition == 1 and t == 0:
                rot_left_b0_grand = mne.grand_average(evoked_list)
                rot_left_b0_flist = evoked_list
            elif task == 0 and condition == 1 and t == 1:
                rot_left_b1_grand = mne.grand_average(evoked_list)
                rot_left_b1_flist = evoked_list
            elif task == 1 and condition == 0 and t == 0:
                mir_right_b0_grand = mne.grand_average(evoked_list)
                mir_right_b0_flist = evoked_list
            elif task == 1 and condition == 0 and t == 1:
                mir_right_b1_grand = mne.grand_average(evoked_list)
                mir_right_b1_flist = evoked_list
            elif task == 1 and condition == 1 and t == 0:
                mir_left_b0_grand = mne.grand_average(evoked_list)
                mir_left_b0_flist = evoked_list
            elif task == 1 and condition == 1 and t == 1:
                mir_left_b1_grand = mne.grand_average(evoked_list)
                mir_left_b1_flist = evoked_list

In [ ]:
#save as evoked objects
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#RANDOM
tasks = ['rdmrot', 'rdmmir']

#split into training blocks
b = list(range(0,48))
training = [b]

#rdm
#only half will have rdm in rot and other half will have rdm in mir, due to us only considering targets along horizontal axis (i.e., vertical mirror)
# for the half in each (rdmrot or rdmmir), half will have early, other half will have late
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for task in range(0, len(tasks)):
        for cond in range(0, len(conditions)):
            diridx, dirtrials = direction_index(tasks[task], conditions[cond], pp)
            diridx = [i for i in diridx if i in training[0]]
            if len(diridx) == 0:
                break
            triallist = trial_list_rdm(tasks[task], pp, diridx)
            pdata = data[triallist]
            print(pp, task, cond, pdata)
            if pp % 2 == 0 and task == 0:
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'rdm', '0')
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'rdm', '0')
            elif pp% 2 == 0 and task == 1:
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'rdm', '1')
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'rdm', '1')
            elif pp% 2 == 1 and task == 0:
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'rdm', '1')
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'rdm', '1')
            elif pp% 2 == 1 and task == 1:
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'rdm', '0')
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'rdm', '0')


# Deprecated -----
# #save as evoked objects
# conditions = ['r', 'l']
# conditionnames = ['rightmoves', 'leftmoves']
# l_cutoff = 30

# #RANDOM
# tasks = ['rdmrot', 'rdmmir']

# #split into training blocks
# b0 = list(range(0,24))
# b1 = list(range(24,48))

# training = [b0, b1]

       
# #rdm
# #only half will have rdm in rot and other half will have rdm in mir, due to us only considering targets along horizontal axis (i.e., vertical mirror)
# for task in range(0, len(tasks)):
#     for pp in participants:
#         data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
#         for cond in range(0, len(conditions)):
#             for t in range(0, len(training)):
#                 diridx, dirtrials = direction_index(tasks[task], conditions[cond], pp)
#                 diridx = [i for i in diridx if i in training[t]]
#                 if len(diridx) == 0:
#                     break
#                 triallist = trial_list_rdm(tasks[task], pp, diridx)
#                 pdata = data[triallist]
#                 print(diridx)
#                 output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'rdm', t)
#                 #filter at 30 Hz
#                 pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
#                 #then average across trials
#                 output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'rdm', t)

In [ ]:
#get grand averages - RANDOM
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']
tasks = ['rdm']
training = [0, 1]
#rdm
for task in range(0, len(tasks)):
    for condition in range(0, len(conditionnames)):
        for t in range(0, len(training)):
            flist = []
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], conditionnames[condition], tasks[task], training[t]))
                if os.path.exists(fname) == False:
                    continue
#                 print(fname)
                evoked = mne.read_evokeds(fname)
                flist.append(evoked)
                evoked = evoked[0]
                evoked_list.append(evoked)
#                 print(evoked_list)
            if task == 0 and condition == 0 and t == 0:
                rdm_right_b0_grand = mne.grand_average(evoked_list)
                rdm_right_b0_flist = evoked_list
            elif task == 0 and condition == 0 and t == 1:
                rdm_right_b1_grand = mne.grand_average(evoked_list)
                rdm_right_b1_flist = evoked_list
            elif task == 0 and condition == 1 and t == 0:
                rdm_left_b0_grand = mne.grand_average(evoked_list)
                rdm_left_b0_flist = evoked_list
            elif task == 0 and condition == 1 and t == 1:
                rdm_left_b1_grand = mne.grand_average(evoked_list)
                rdm_left_b1_flist = evoked_list

In [ ]:
#runs for LRP ONLY: need separate files for C3 and C4

flists = ['aln_right', 'aln_left', 'alnrot_right', 'alnrot_left', 'alnmir_right', 'alnmir_left',
          'alnearlyrdm_right', 'alnearlyrdm_left', 'alnlaterdm_right', 'alnlaterdm_left',
          'rot_right_b0', 'rot_right_b1', 'rot_left_b0', 'rot_left_b1',
          'mir_right_b0', 'mir_right_b1', 'mir_left_b0', 'mir_left_b1',
          'rdm_right_b0', 'rdm_right_b1', 'rdm_left_b0', 'rdm_left_b1']

channels = ['C3', 'C4']
for channel in channels:
    for f in flists:
        newdf = pd.DataFrame()
        if f == 'aln_right':
            for pp in range(0, len(aln_right_flist)):
                df = aln_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'aln_left':
            for pp in range(0, len(aln_left_flist)):
                df = aln_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnrot_right':
            for pp in range(0, len(alnrot_right_flist)):
                df = alnrot_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnrot_left':
            for pp in range(0, len(alnrot_left_flist)):
                df = alnrot_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnmir_right':
            for pp in range(0, len(alnmir_right_flist)):
                df = alnmir_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnmir_left':
            for pp in range(0, len(alnmir_left_flist)):
                df = alnmir_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnearlyrdm_right':
            for pp in range(0, len(alnearlyrdm_right_flist)):
                df = alnearlyrdm_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnearlyrdm_left':
            for pp in range(0, len(alnearlyrdm_left_flist)):
                df = alnearlyrdm_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnlaterdm_right':
            for pp in range(0, len(alnlaterdm_right_flist)):
                df = alnlaterdm_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnlaterdm_left':
            for pp in range(0, len(alnlaterdm_left_flist)):
                df = alnlaterdm_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_b0':
            for pp in range(0, len(rot_right_b0_flist)):
                df = rot_right_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_b1':
            for pp in range(0, len(rot_right_b1_flist)):
                df = rot_right_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_left_b0':
            for pp in range(0, len(rot_left_b0_flist)):
                df = rot_left_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_left_b1':
            for pp in range(0, len(rot_left_b1_flist)):
                df = rot_left_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_right_b0':
            for pp in range(0, len(mir_right_b0_flist)):
                df = mir_right_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_right_b1':
            for pp in range(0, len(mir_right_b1_flist)):
                df = mir_right_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_left_b0':
            for pp in range(0, len(mir_left_b0_flist)):
                df = mir_left_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_left_b1':
            for pp in range(0, len(mir_left_b1_flist)):
                df = mir_left_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_right_b0':
            for pp in range(0, len(rdm_right_b0_flist)):
                df = rdm_right_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_right_b1':
            for pp in range(0, len(rdm_right_b1_flist)):
                df = rdm_right_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_left_b0':
            for pp in range(0, len(rdm_left_b0_flist)):
                df = rdm_left_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_left_b1':
            for pp in range(0, len(rdm_left_b1_flist)):
                df = rdm_left_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
             


In [ ]:
# # Difference waves between aligned and perturbed conditions for C3
# #channels = ['C3']
# channels = ['C4']
# # channels = ['Oz','O1','O2']
# evokeds = dict(aln_right = right_aln_flist, 
#                aln_left = left_aln_flist,
#                rot_right = right_rot_flist,
#                rot_left = left_rot_flist,
#                rdm_right = right_rdm_flist,
#                rdm_left = left_rdm_flist,
#                mir_right = right_mir_flist,
#                mir_left = left_mir_flist)
# #plot
# mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)